In [1]:
import pandas as pd
import numpy as np
import re

# --- Config ---
INPUT_PATH = "donnees_brvm_large.xlsx"   # ou .csv
SHEET_NAME = 0                           # si Excel : feuille à lire
OUTPUT_PATH = "donnees_brvm_long.csv"

# Les variables (suffixes) attendues après le ticker
MEASURES = [
    "CA_PNB", "Total Bilan", "RE_RBE", "Résultat Net", "Capitaux propres"
]

# --- 1) Charger ---
# Si ton fichier est un CSV, remplace par: df = pd.read_csv(INPUT_PATH)
df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME)

# On garde ce qui est dispo parmi ces colonnes d'identification
id_vars = [c for c in ["Période_détail", "Période"] if c in df.columns]
if not id_vars:
    raise ValueError("Aucune des colonnes 'Période_détail' ou 'Période' n'a été trouvée.")

# --- 2) Passer en long (melt) puis extraire Ticker et Mesure ---
melted = df.melt(id_vars=id_vars, var_name="colonne", value_name="valeur")

# Regex: "tout ce qu'il y a avant le dernier '_' = Ticker", puis l'un des suffixes cibles
pattern = rf"^(.*?)_({'|'.join(map(re.escape, MEASURES))})$"
extracted = melted["colonne"].str.extract(pattern)

melted["Ticker"] = extracted[0]
melted["Mesure"] = extracted[1]
melted = melted.dropna(subset=["Ticker", "Mesure"]).drop(columns=["colonne"])

# --- 3) Repasser en large par Mesure (colonnes) ---
long_table = (
    melted
    .pivot_table(index=id_vars + ["Ticker"], columns="Mesure", values="valeur", aggfunc="first")
    .reset_index()
)

# Les colonnes peuvent devenir un MultiIndex après pivot ; on les aplati si besoin
long_table.columns.name = None

# --- 4) Nettoyage des montants -> nombres (en supprimant "CFA", espaces, etc.) ---
def to_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    # Remplacer toutes les virgules par des points (si présent)
    s = s.replace(",", ".")
    # Supprimer tout sauf chiffres, signe, point et virgule déjà gérée
    s = re.sub(r"[^\d\.\-]", "", s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

for m in MEASURES:
    if m in long_table.columns:
        long_table[m] = long_table[m].apply(to_number)

# --- 5) Mise en forme finale ---
# Renommer pour coller exactement à ta demande
rename_map = {
    "Période": "Période",
    "Période_détail": "Période_détail",
}
long_table = long_table.rename(columns=rename_map)

# Ordonner les colonnes de sortie
cols_out = [c for c in ["Ticker", "Période", "Période_détail"] if c in long_table.columns] + MEASURES
# On remet les colonnes présentes seulement (au cas où certaines mesures manquent)
cols_out = [c for c in cols_out if c in long_table.columns]
long_table = long_table[cols_out]

# --- 6) Sauvegarder ---
long_table.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
print(f"OK -> {OUTPUT_PATH}")


OK -> donnees_brvm_long.csv
